In [1]:
import pandas as pd

In [2]:
year = 2019
crime_df = pd.read_csv('CrimeData{}-withlocality.csv'.format(year))[['From_Date', 'nbh_name']].dropna()
crime_df.head()
print(len(crime_df))

67735


In [3]:
crime_df['month'] = crime_df['From_Date'].apply(lambda x: x.split('/')[0])
crime_df.head()

,From_Date,nbh_name,month
0,03/18/2019,Line Creek And Northern Heights,03
3,03/24/2019,Winnetonka,03
4,03/25/2019,Crown Center,03
6,03/25/2019,Southmoreland,03
8,03/26/2019,East Meyer 6,03


In [4]:
crime_df = crime_df.groupby(['nbh_name', 'month']).size().to_frame('count').reset_index()

In [5]:
import pandas as pd
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from tqdm.notebook import tqdm
scaled_df = pd.DataFrame()
for nbh in tqdm(crime_df['nbh_name'].unique(), total = len(crime_df['nbh_name'].unique())):
    sub_df = crime_df.loc[crime_df['nbh_name']==nbh]
    x = sub_df['count'].values.reshape(-1, 1)
#     display(x)
    scaler = StandardScaler()
    x_scaled = scaler.fit_transform(x)
    df = pd.DataFrame(x_scaled, columns = ['count_scaled'])
#     display(df)
#     print(x_scaled.std(axis = 0))
#     print(np.mean(x_scaled))
    sub_df['label'] = pd.cut(df['count_scaled'], bins=[-100, -0.5, 0.5, 100 ], include_lowest=True, labels = ['low', 'avg', 'high']).values
    sub_df['frequency'] = x_scaled 
#     display(sub_df)
    
    
    scaled_df = scaled_df.append(sub_df, ignore_index = True)
    
#     break
scaled_df

  0%|          | 0/240 [00:00<?, ?it/s]

C:\Users\dhh3hb\Anaconda3\envs\nsfscc-pytorch\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\dhh3hb\Anaconda3\envs\nsfscc-pytorch\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\dhh3hb\Anaconda3\envs\nsfscc-pytorch\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See t

,nbh_name,month,count,label,frequency
0,18th And Vine And Downtown East,03,8,low,-2.266087
1,18th And Vine And Downtown East,04,41,avg,-0.027139
2,18th And Vine And Downtown East,05,45,avg,0.244249
3,18th And Vine And Downtown East,06,57,high,1.058412
4,18th And Vine And Downtown East,07,47,avg,0.379943
...,...,...,...,...,...
2435,Wornall Homestead,05,2,low,-0.529999
2436,Wornall Homestead,06,1,low,-1.165998
2437,Wornall Homestead,07,3,avg,0.106000
2438,Wornall Homestead,08,6,high,2.013996


In [6]:
scaled_df[100:]

,nbh_name,month,count,label,frequency
100,Blenheim Square Research Hospital,11,71,avg,-0.090394
101,Blenheim Square Research Hospital,12,60,low,-0.522711
102,Blue Hills,01,5,low,-2.251428
103,Blue Hills,03,38,low,-1.535781
104,Blue Hills,04,136,high,0.589472
...,...,...,...,...,...
2435,Wornall Homestead,05,2,low,-0.529999
2436,Wornall Homestead,06,1,low,-1.165998
2437,Wornall Homestead,07,3,avg,0.106000
2438,Wornall Homestead,08,6,high,2.013996


In [7]:
scaled_df[['month','nbh_name', 'label']].astype('category')
scaled_df.head()

,nbh_name,month,count,label,frequency
0,18th And Vine And Downtown East,03,8,low,-2.266087
1,18th And Vine And Downtown East,04,41,avg,-0.027139
2,18th And Vine And Downtown East,05,45,avg,0.244249
3,18th And Vine And Downtown East,06,57,high,1.058412
4,18th And Vine And Downtown East,07,47,avg,0.379943


In [8]:
from sklearn.preprocessing import LabelEncoder
# creating instance of labelencoder
month_labelencoder = LabelEncoder()
nbh_labelencoder = LabelEncoder()
intensity_labelencoder = LabelEncoder()

# Assigning numerical values and storing in another column
scaled_df['month'] = month_labelencoder.fit_transform(scaled_df['month'])
scaled_df['nbh_name'] = nbh_labelencoder.fit_transform(scaled_df['nbh_name'])
scaled_df['label'] = intensity_labelencoder.fit_transform(scaled_df['label'])

scaled_df

,nbh_name,month,count,label,frequency
0,0,2,8,2,-2.266087
1,0,3,41,0,-0.027139
2,0,4,45,0,0.244249
3,0,5,57,1,1.058412
4,0,6,47,0,0.379943
...,...,...,...,...,...
2435,239,4,2,2,-0.529999
2436,239,5,1,2,-1.165998
2437,239,6,3,0,0.106000
2438,239,7,6,1,2.013996


In [9]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

X = scaled_df[['nbh_name','month']]
y = scaled_df[['frequency', 'label']]
# y2 = crime_df['label']

# Train - Test
X_trainval, X_test, y_trainval, y_test = train_test_split(X, y, test_size=0.2, random_state=69)
# Split train" into train-val
X_train, X_val, y_train, y_val = train_test_split(X_trainval, y_trainval, test_size=0.1, random_state=21)


In [10]:
X_train, y_train = np.array(X_train), np.array(y_train)
X_val, y_val = np.array(X_val), np.array(y_val)
X_test, y_test = np.array(X_test), np.array(y_test)

In [11]:
X_train.shape

(1756, 2)

In [12]:
from torch.utils.data import Dataset, DataLoader

class RegressionDataset(Dataset):
    
    def __init__(self, X_data, y_data):
        self.X_data = X_data
        self.y_data = y_data
        
    def __getitem__(self, index):
        return self.X_data[index], self.y_data[index]
        
    def __len__ (self):
        return len(self.X_data)
train_dataset = RegressionDataset(torch.from_numpy(X_train).float(), torch.from_numpy(y_train).float())
val_dataset = RegressionDataset(torch.from_numpy(X_val).float(), torch.from_numpy(y_val).float())
test_dataset = RegressionDataset(torch.from_numpy(X_test).float(), torch.from_numpy(y_test).float())
# unseen_dataset = RegressionDataset(torch.from_numpy(Z_test).float(), torch.from_numpy(y_test).float())

In [13]:
EPOCHS = 10
BATCH_SIZE = 64
LEARNING_RATE = 0.0001
NUM_FEATURES = len(X.columns)
# NUM_FEATURES = 1
train_loader = DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(dataset=val_dataset, batch_size=1)
test_loader = DataLoader(dataset=test_dataset, batch_size=1)
# unseen_loader = DataLoader(dataset=unseen_dataset, batch_size=1)

In [14]:
# class ClassificationRegression(nn.Module):
#     def __init__(self, num_features):
#         super(MultipleRegression, self).__init__()
#         self.layer_1 = nn.Linear(num_features, 32)
#         self.layer_2 = nn.Linear(32, 64)
#         self.layer_3 = nn.Linear(64, 128)
#         self.layer_4 = nn.Linear(128, 256)
#         self.layer_5 = nn.Linear(256, 256)
#         self.layer_6 = nn.Linear(256, 128)
#         self.layer_7 = nn.Linear(128, 64)
#         self.layer_8 = nn.Linear(64, 32)
#         self.layer_9 = nn.Linear(32, 16)
#         self.layer_out = nn.Linear(16, 1)
#         self.relu = nn.ReLU()
#         self.relu2 = nn.LeakyReLU()

#     def forward(self, inputs):
#       x = self.relu(self.layer_1(inputs))
#       x = self.relu2(self.layer_2(x))
#       x = self.relu(self.layer_3(x))
#       x = self.relu2(self.layer_4(x))
#       x = self.relu(self.layer_5(x))
#       x = self.relu2(self.layer_6(x))
#       x = self.relu(self.layer_7(x))
#       x = self.relu2(self.layer_8(x))
#       x = self.relu(self.layer_9(x))
#       x = self.layer_out(x)
#       return (x)
#     def predict(self, test_inputs):
#       x = self.relu(self.layer_1(test_inputs))
#       x = self.relu2(self.layer_2(x))
#       x = self.relu(self.layer_3(x))
#       x = self.relu2(self.layer_4(x))
#       x = self.relu(self.layer_5(x))
#       x = self.relu2(self.layer_6(x))
#       x = self.relu(self.layer_7(x))
#       x = self.relu2(self.layer_8(x))
#       x = self.relu(self.layer_9(x))
#       x = self.layer_out(x)
#       return (x)


class ClassiRegression(nn.Module):
    def __init__(self, num_features):
        super(ClassiRegression, self).__init__()
        
        self.layer_1 = nn.Linear(num_features, 16)
        self.layer_2 = nn.Linear(16, 32)
        self.layer_3 = nn.Linear(32, 16)
        self.layer_out1 = nn.Linear(16, 1)
        self.layer_out2 = nn.Linear(16, 1)
        
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
    def forward(self, inputs):
            x = self.relu(self.layer_1(inputs))
            x = self.relu(self.layer_2(x))
            
            x1 = self.relu(self.layer_3(x))
            x1 = self.layer_out1(x1)
            
            x2 = self.sigmoid(self.layer_3(x))
            x2 = self.layer_out2(x2)
            return x1,x2
    def predict(self, test_inputs):
            x = self.relu(self.layer_1(test_inputs))
            x = self.relu(self.layer_2(x))
            
            x1 = self.relu(self.layer_3(x))
            x1 = self.layer_out1(x1)
            
            x2 = self.sigmoid(self.layer_3(x))
            x2 = self.layer_out2(x2)
            return x1,x2
        
# class ClassiRegression(nn.Module):
#     def __init__(self, num_features):
#         super(ClassiRegression, self).__init__()
        
#         self.layer_1 = nn.Linear(num_features, 16)
#         self.layer_2 = nn.Linear(16, 32)
#         self.layer_3 = nn.Linear(32, 16)
#         self.layer_out1 = nn.Linear(16, 1)
#         self.layer_out2 = nn.Linear(16, 1)
        
#         self.relu = nn.ReLU()
#         self.sigmoid = nn.Sigmoid()
#     def forward(self, inputs):
#             x = self.relu(self.layer_1(inputs))
#             x = self.relu(self.layer_2(x))
            
#             x1 = self.relu(self.layer_3(x))
#             x1 = self.layer_out1(x1)
            

#             return x1
#     def predict(self, test_inputs):
#             x = self.relu(self.layer_1(test_inputs))
#             x = self.relu(self.layer_2(x))
            
#             x1 = self.relu(self.layer_3(x))
#             x1 = self.layer_out1(x1)
            
#             return x1

In [21]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
model = ClassiRegression(NUM_FEATURES)
model.to(device)
print(model)
criterion1 = nn.MSELoss()
criterion2 = nn.CrossEntropyLoss()

optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)


cuda:0


RuntimeError: CUDA error: device-side assert triggered

In [16]:
from torchsummary import summary
summary(model, (2,2))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                [-1, 2, 16]              48
              ReLU-2                [-1, 2, 16]               0
            Linear-3                [-1, 2, 32]             544
              ReLU-4                [-1, 2, 32]               0
            Linear-5                [-1, 2, 16]             528
              ReLU-6                [-1, 2, 16]               0
            Linear-7                 [-1, 2, 1]              17
            Linear-8                [-1, 2, 16]             528
           Sigmoid-9                [-1, 2, 16]               0
           Linear-10                 [-1, 2, 1]              17
Total params: 1,682
Trainable params: 1,682
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.01
Estimated Total

In [ ]:
loss_stats = {
    'train': [],
    "val": []
}
print("Begin training.")
for e in tqdm(range(1, EPOCHS+1)):
    
    # TRAINING
    train_epoch_loss = 0
    model.train()
    for X_train_batch, y_train_batch in train_loader:
        X_train_batch, y_train_batch = X_train_batch.to(device), y_train_batch.to(device)
        
        optimizer.zero_grad()
        target_y1 = y_train_batch[:, 0].reshape(-1,1)
        
        y_train_pred = model(X_train_batch)
        train_loss = criterion(y_train_pred, target_y1)
        # print(y_train_pred, y_train_batch)
        train_loss.backward()
        optimizer.step()
        
        train_epoch_loss += train_loss.item()
        
        
    # VALIDATION    
    with torch.no_grad():
        
        val_epoch_loss = 0
        
        model.eval()
        for X_val_batch, y_val_batch in val_loader:
            X_val_batch, y_val_batch = X_val_batch.to(device), y_val_batch.to(device)
            
            y_val_pred = model(X_val_batch)
                        
            val_loss = criterion(y_val_pred, y_val_batch)
            
            val_epoch_loss += val_loss.item()
        loss_stats['train'].append(train_epoch_loss/len(train_loader))
        loss_stats['val'].append(val_epoch_loss/len(val_loader))                              
            
        print(f'Epoch {e+0:03}: | Train Loss: {train_epoch_loss/len(train_loader):.5f} | Val Loss: {val_epoch_loss/len(val_loader):.5f}')

In [19]:
loss_stats = {
    'train': [],
    "val": []
}
print("Begin training.")
for e in tqdm(range(1, EPOCHS+1)):
    
    # TRAINING
    train_epoch_loss = 0
    model.train()
    for X_train_batch, y_train_batch in train_loader:
        X_train_batch, y_train_batch = X_train_batch.to(device), y_train_batch.to(device)
        optimizer.zero_grad()
        
        out1, out2 = model(X_train_batch)
#         print(out1)
        target_y1 = y_train_batch[:, 0].reshape(-1,1)
        target_y2 = y_train_batch[:, 1].reshape(-1,1).long().flatten()
#         print(target_y1)
#         print(out2, target_y2)
        train_loss_1 = criterion1(out1, target_y1)
#         train_loss_2 = criterion2(out2, target_y2)
        alpha = 0.6 # define the weights of each sub-loss in the global loss
#         train_loss = alpha * train_loss_1 + (1-alpha) * train_loss_2
        
        train_loss = train_loss_1
        print(train_loss_1, train_loss_2, train_loss)
#         train_loss.backward()
#         optimizer.step()
        
#         train_epoch_loss += train_loss.item()
        
        break
#         
        
#     # VALIDATION    
#     with torch.no_grad():
        
#         val_epoch_loss = 0
        
#         model.eval()
#         for X_val_batch, y_val_batch in val_loader:
#             X_val_batch, y_val_batch = X_val_batch.to(device), y_val_batch.to(device)
            
#             y_val_pred = model(X_val_batch)
                        
#             val_loss = criterion(y_val_pred, y_val_batch)
            
#             val_epoch_loss += val_loss.item()
#         loss_stats['train'].append(train_epoch_loss/len(train_loader))
#         loss_stats['val'].append(val_epoch_loss/len(val_loader))                              
            
#         print(f'Epoch {e+0:03}: | Train Loss: {train_epoch_loss/len(train_loader):.5f} | Val Loss: {val_epoch_loss/len(val_loader):.5f}')

    break

Begin training.


  0%|          | 0/10 [00:00<?, ?it/s]

RuntimeError: CUDA error: device-side assert triggered